In [1]:
from numpy import genfromtxt
import tensorflow as tf
import tensorflow.keras as keras
'''#from keras.models import Sequential
from keras.layers import Dense, Input, Lambda, Dropout
from keras import regularizers
#from keras.layers import Dropout
from keras.constraints import maxnorm
#from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Model
from keras import regularizers
#from keras.layers.normalization import BatchNormalization
from keras import backend as K
from keras import objectives
from keras.models import model_from_json
from keras.layers import Input, Dense, Convolution1D, MaxPooling1D, UpSampling1D, AveragePooling1D, Flatten ,Reshape,ZeroPadding1D,Cropping1D
from keras.models import Model
#from keras.callbacks import TensorBoard
from keras.regularizers import l2, l1
from keras import optimizers
from keras.layers.noise import GaussianNoise
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import optimizers
from keras.layers import Dense,concatenate, Input,TimeDistributed,LSTM, Lambda, Dropout, Reshape, Conv1D, Flatten, UpSampling1D, Flatten, BatchNormalization
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard
from keras import backend as K'''

import tensorflow.keras.backend as K

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape, LSTM, Lambda
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import tensorflow as tf


from PIL import Image

import os
import sys
import itertools
#import keras_internal_model
import pandas as pd
import numpy as np
#import train_tools as tt
from numpy import genfromtxt
from time import time

import csv

tf.compat.v1.enable_v2_behavior()


gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(gpus[0], 
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5000)])
  except RuntimeError as e:
    print(e)
    
    
print("GPUs: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.experimental.list_logical_devices('GPU')

import cv2
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from sklearn import preprocessing

GPUs:  2


In [2]:
def load_images(parent_path,imgs_paths):
    imgs = []
    for i in imgs_paths:
        imgs.append( cv2.imread( os.path.join(parent_path,i)) )
    return np.array(imgs)

In [3]:
img_path = '/vol/fob-vol6/mi13/pivillaa/uni/romi/rl_sony/best_next_move_3d_reconstruction/data/imgs'
idx_file = '/vol/fob-vol6/mi13/pivillaa/uni/romi/rl_sony/best_next_move_3d_reconstruction/data/samples_idx.csv'
#img_path = '/home/pico/uni/romi/rl_sony/best_next_move_3d_reconstruction/data/imgs'
#idx_file = '/home/pico/uni/romi/rl_sony/best_next_move_3d_reconstruction/data/samples_idx.csv'

df = pd.read_csv(idx_file,usecols=(0,5,6))

In [4]:
imgs = load_images(img_path,df['img'])

In [5]:
print(imgs.shape)
df.head()

(25000, 82, 82, 3)


,img,dp2,dt2
0,009_2d/657_116_281.png,-1,-20
1,001_2d/600_580_044.png,0,-20
2,006_2d/146_686_681.png,-3,-20
3,003_2d/511_158_511.png,-2,-13
4,001_2d/521_535_546.png,-3,20


In [6]:
y = df[['dp2','dt2']].values

scaler = preprocessing.StandardScaler().fit(y)
y = scaler.transform(y)

In [7]:
th = 17500
x_train = imgs[:th]
y_train = y[:th]
x_test = imgs[th:]
y_test = y[th:]

x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")


x_train = x_train[:1000]
x_test = x_test[:1000]

y_train = y_train[:1000]
y_test = y_test[:1000]

In [8]:
def conv_layer(d, k):
    """ Returns a 2D Conv layer, with and ReLU activation
    """
    # return Conv2D(d, k, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')
    return Conv2D(d, k, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')
def conv_block(inp, d=3, pool_size=(2, 2), k=3):
    """ Returns a 2D Conv block, with a convolutional layer, max-pooling
    """
    conv = conv_layer(d, k)(inp)
    return MaxPooling2D(pool_size=pool_size)(conv)

'''def network():
    inp = Input((82,82,3))
    x = conv_block(inp, 32, (2, 2), 8)
    x = conv_block(x, 64, (2, 2), 4)
    x = conv_block(x, 64, (2, 2), 3)
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    x = Dense(2, activation='linear')(x)
    return Model(inp, x)'''

def network():
    # Network defined by the Deepmind paper
    inputs = Input(shape=(82, 82, 3,))

    # Convolutions on the frames on the screen
    layer1 = Conv2D(32, 8, strides=1, activation="relu")(inputs)
    layer2 = Conv2D(64, 3, strides=1, activation="relu")(layer1)
    layer3 = Conv2D(64, 3, strides=1, activation="relu")(layer2)

    layer4 = Flatten()(layer3)

    layer5 = Dense(512, activation="relu")(layer4)
    action = Dense(2, activation="linear")(layer5)

    return Model(inputs=inputs, outputs=action)

In [9]:
model = network()

In [10]:
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt, 
            loss=tf.keras.losses.MeanSquaredError())#,
            #metrics=[tf.keras.metrics.MeanSquaredError()])

In [11]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./generated_data/logs",
                                                        histogram_freq=1,
                                                        write_images=True)

In [12]:
history = model.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=1000,
    validation_data=(x_test, y_test),
    callbacks=[tensorboard_callback]
)

Epoch 1/1000
16/16 [==============================] - 5s 154ms/step - loss: 46.1451 - val_loss: 1.0117
Epoch 2/1000
16/16 [==============================] - 1s 38ms/step - loss: 1.0175 - val_loss: 1.0098
Epoch 3/1000
16/16 [==============================] - 1s 38ms/step - loss: 1.0320 - val_loss: 1.0120
Epoch 4/1000
16/16 [==============================] - 1s 40ms/step - loss: 1.0160 - val_loss: 1.0124
Epoch 5/1000
16/16 [==============================] - 1s 39ms/step - loss: 1.0122 - val_loss: 1.0126
Epoch 6/1000
16/16 [==============================] - 1s 39ms/step - loss: 1.0149 - val_loss: 1.0129
Epoch 7/1000
16/16 [==============================] - 1s 39ms/step - loss: 1.0527 - val_loss: 1.0133
Epoch 8/1000
16/16 [==============================] - 1s 39ms/step - loss: 1.0176 - val_loss: 1.0135
Epoch 9/1000
16/16 [==============================] - 1s 40ms/step - loss: 1.0277 - val_loss: 1.0141
Epoch 10/1000
16/16 [==============================] - 1s 38ms/step - loss: 1.0114 - val_

16/16 [==============================] - 1s 37ms/step - loss: 1.0577 - val_loss: 1.0200
Epoch 82/1000
16/16 [==============================] - 1s 38ms/step - loss: 1.0133 - val_loss: 1.0203
Epoch 83/1000
16/16 [==============================] - 1s 38ms/step - loss: 0.9993 - val_loss: 1.0199
Epoch 84/1000
16/16 [==============================] - 1s 37ms/step - loss: 1.0221 - val_loss: 1.0204
Epoch 85/1000
16/16 [==============================] - 1s 39ms/step - loss: 1.0126 - val_loss: 1.0199
Epoch 86/1000
16/16 [==============================] - 1s 38ms/step - loss: 1.0284 - val_loss: 1.0195
Epoch 87/1000
16/16 [==============================] - 1s 38ms/step - loss: 0.9984 - val_loss: 1.0193
Epoch 88/1000
16/16 [==============================] - 1s 38ms/step - loss: 1.0385 - val_loss: 1.0195
Epoch 89/1000
16/16 [==============================] - 1s 38ms/step - loss: 0.9882 - val_loss: 1.0194
Epoch 90/1000
16/16 [==============================] - 1s 38ms/step - loss: 0.9939 - val_loss: 1

Epoch 161/1000
16/16 [==============================] - 1s 37ms/step - loss: 1.0164 - val_loss: 1.0197
Epoch 162/1000
16/16 [==============================] - 1s 36ms/step - loss: 1.0070 - val_loss: 1.0201
Epoch 163/1000
16/16 [==============================] - 1s 38ms/step - loss: 1.0024 - val_loss: 1.0194
Epoch 164/1000
16/16 [==============================] - 1s 36ms/step - loss: 1.0268 - val_loss: 1.0194
Epoch 165/1000
16/16 [==============================] - 1s 37ms/step - loss: 1.0568 - val_loss: 1.0198
Epoch 166/1000
16/16 [==============================] - 1s 38ms/step - loss: 1.0315 - val_loss: 1.0203
Epoch 167/1000
16/16 [==============================] - 1s 37ms/step - loss: 1.0069 - val_loss: 1.0198
Epoch 168/1000
16/16 [==============================] - 1s 37ms/step - loss: 1.0315 - val_loss: 1.0196
Epoch 169/1000
16/16 [==============================] - 1s 38ms/step - loss: 1.0084 - val_loss: 1.0192
Epoch 170/1000
16/16 [==============================] - 1s 37ms/step - lo

16/16 [==============================] - 1s 38ms/step - loss: 1.0268 - val_loss: 1.0189
Epoch 241/1000
16/16 [==============================] - 1s 37ms/step - loss: 1.0160 - val_loss: 1.0203
Epoch 242/1000
16/16 [==============================] - 1s 37ms/step - loss: 1.0068 - val_loss: 1.0197
Epoch 243/1000
16/16 [==============================] - 1s 36ms/step - loss: 1.0147 - val_loss: 1.0197
Epoch 244/1000
16/16 [==============================] - 1s 38ms/step - loss: 1.0357 - val_loss: 1.0192
Epoch 245/1000
16/16 [==============================] - 1s 36ms/step - loss: 0.9839 - val_loss: 1.0198
Epoch 246/1000
16/16 [==============================] - 1s 37ms/step - loss: 1.0089 - val_loss: 1.0197
Epoch 247/1000
16/16 [==============================] - 1s 37ms/step - loss: 1.0291 - val_loss: 1.0203
Epoch 248/1000
16/16 [==============================] - 1s 37ms/step - loss: 1.0077 - val_loss: 1.0194
Epoch 249/1000
16/16 [==============================] - 1s 37ms/step - loss: 1.0184 - va

16/16 [==============================] - 1s 37ms/step - loss: 1.0348 - val_loss: 1.0197
Epoch 320/1000
16/16 [==============================] - 1s 37ms/step - loss: 1.0099 - val_loss: 1.0191
Epoch 321/1000
16/16 [==============================] - 1s 36ms/step - loss: 1.0457 - val_loss: 1.0196
Epoch 322/1000
16/16 [==============================] - 1s 37ms/step - loss: 1.0192 - val_loss: 1.0190
Epoch 323/1000
16/16 [==============================] - 1s 36ms/step - loss: 1.0262 - val_loss: 1.0194
Epoch 324/1000
16/16 [==============================] - 1s 37ms/step - loss: 1.0075 - val_loss: 1.0194
Epoch 325/1000
16/16 [==============================] - 1s 37ms/step - loss: 1.0041 - val_loss: 1.0193
Epoch 326/1000
16/16 [==============================] - 1s 37ms/step - loss: 1.0391 - val_loss: 1.0195
Epoch 327/1000
16/16 [==============================] - 1s 36ms/step - loss: 0.9862 - val_loss: 1.0197
Epoch 328/1000
16/16 [==============================] - 1s 37ms/step - loss: 1.0360 - va

KeyboardInterrupt: 

In [ ]:
#plt.imshow(x_train[0,:,:,2])

In [ ]:
x_train.shape